In [62]:
import nltk
import string

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# preprocessing documents
def preprocess(document):
    # 1. Lowercase Sentence
    document = document.lower()

    # 2. Sentence Splitter
    bag_of_document = document.split(' ')

    # 3. Punctuation Remove
    for i, word in enumerate(bag_of_document):
            bag_of_document[i] = word.translate(str.maketrans('', '', string.punctuation))

    # 4. Stop Word Remove
    #print(stopwords.words('english'))
    bag_of_document = [word for word in bag_of_document if word not in stopwords.words('english')]

    # 5. Lemmatize
    for i, word in enumerate(bag_of_document):
            bag_of_document[i] = lemmatizer.lemmatize(word)

    bag_of_document = ' '.join(bag_of_document)

    return bag_of_document

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kilometers/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kilometers/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/kilometers/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [63]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('Reviews.csv')
df.drop(['UserId', 'ProductId', 'Id', 'ProfileName','HelpfulnessDenominator', 'HelpfulnessNumerator', 'Time', 'Summary'], axis=1, inplace=True)

In [64]:
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    df.at[idx, 'Text'] = preprocess(row['Text'])

100%|██████████| 568454/568454 [1:53:13<00:00, 83.68it/s]  


In [65]:
df.to_csv('Output.csv')

In [69]:
df = pd.read_csv('Output.csv')

df['Rating'] = df.Score.apply(lambda x: 1 if x in [3, 4, 5] else 0)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.head()

,Score,Text,Rating
0,5,bought several vitality canned dog food produc...,1
1,1,product arrived labeled jumbo salted peanutsth...,0
2,4,confection around century light pillowy citru...,1
3,2,looking secret ingredient robitussin believe f...,0
4,5,great taffy great price wide assortment yummy...,1


In [75]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 40000
embedding_dim = 16
max_length = 120

split = round(len(df)*0.7)

train_text = df['Text'][:split]
train_rating = df['Rating'][:split]

test_text = df['Text'][split:]
test_rating = df['Rating'][split:]

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_text)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(train_text)
padded = pad_sequences(sequences, maxlen=max_length, truncating='post')

testing_sentences = tokenizer.texts_to_sequences(test_text)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)


In [77]:
import tensorflow as tf

rnn_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

num_epochs = 20
history = rnn_model.fit(padded, train_rating, epochs=num_epochs, steps_per_epoch=1000,validation_data=(testing_padded, test_rating))

Epoch 1/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3804 - accuracy: 0.8617 - val_loss: 0.2464 - val_accuracy: 0.8993
Epoch 2/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2193 - accuracy: 0.9120 - val_loss: 0.2105 - val_accuracy: 0.9159
Epoch 3/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1974 - accuracy: 0.9229 - val_loss: 0.2019 - val_accuracy: 0.9220
Epoch 4/20
1000/1000 [==============================] - 10s 10ms/step - loss: 0.1866 - accuracy: 0.9279 - val_loss: 0.1981 - val_accuracy: 0.9239
Epoch 5/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1789 - accuracy: 0.9310 - val_loss: 0.1961 - val_accuracy: 0.9247
Epoch 6/20
1000/1000 [==============================] - 8s 8ms/step - loss: 0.1726 - accuracy: 0.9337 - val_loss: 0.1946 - val_accuracy: 0.9259
Epoch 7/20
1000/1000 [==============================] - 8s 8ms/step - loss: 0.1671 - accuracy: 0.9364 - val_loss: 0.1943 - val_accurac

In [79]:
lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

num_epochs = 20
history = lstm_model.fit(padded, train_rating, epochs=num_epochs, steps_per_epoch=1000,validation_data=(testing_padded, test_rating))

Epoch 1/20
1000/1000 [==============================] - 211s 207ms/step - loss: 0.2539 - accuracy: 0.9035 - val_loss: 0.1913 - val_accuracy: 0.9240
Epoch 2/20
1000/1000 [==============================] - 211s 210ms/step - loss: 0.1734 - accuracy: 0.9318 - val_loss: 0.1824 - val_accuracy: 0.9281
Epoch 3/20
1000/1000 [==============================] - 204s 204ms/step - loss: 0.1537 - accuracy: 0.9399 - val_loss: 0.1829 - val_accuracy: 0.9302
Epoch 4/20
1000/1000 [==============================] - 206s 206ms/step - loss: 0.1366 - accuracy: 0.9472 - val_loss: 0.1851 - val_accuracy: 0.9311
Epoch 5/20
1000/1000 [==============================] - ETA: 0s - loss: 0.1220 - accuracy: 0.9524